In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("data/FILTERED_Police_Traffic_Crash_Reports.csv")
data.head()

,Accident_Date,Accident_Time,At_Intersection,CASE_Number,City,Day_of_Week,Intersection_Type,Light_Conditions,Location_of_First_Harmful_Event,Main_Street,...,Type_of_Collision,Weather_Condition,Work_Zone_Location,Work_Zone_Related,Work_Zone_Type,Workers_Present_in_Work_Zone,Zone_ID,OBJECTID,latitude,longitude
0,2020/01/03 19:35:00+00,1935,100 Feet From,2.020000e+09,VIRGINIA BEACH,FRI,NOT AT INTERSECTION,DARKNESS-ROAD LIGHTED,ON ROADWAY,1600 BLK GENERAL BOOTH BOULEVARD,...,ANGLE,RAIN,NaN,NO,NaN,NaN,na,1,36.785572,-75.998705
1,2020/01/04 00:00:00+00,1505,Yes,2.020000e+09,VIRGINIA BEACH,SAT,NOT AT INTERSECTION,DAYLIGHT,ON ROADWAY,5670 INDIAN RIVER RD,...,SIDESWIPE-SAME DIRECTION,RAIN,NaN,NO,NaN,NaN,na,2,36.805590,-76.190396
2,2020/01/06 00:00:00+00,815,50 Feet From,2.020001e+09,VIRGINIA BEACH,MON,NOT AT INTERSECTION,DAYLIGHT,ON ROADWAY,700 GREAT NECK RD,...,REAR END,NO ADVERSE CONDITION (CLEAR/CLOUDY),NaN,NO,NaN,NaN,na,3,36.852291,-76.048317
3,2020/01/07 19:15:00+00,1915,150 Feet From,2.020001e+09,VIRGINIA BEACH,TUE,FOUR APPROACHES,DARKNESS-ROAD LIGHTED,ON ROADWAY,100 BLOCK LONDON BRIDGE,...,SIDESWIPE-SAME DIRECTION,NO ADVERSE CONDITION (CLEAR/CLOUDY),NaN,NO,NaN,NaN,na,4,36.841992,-76.049118
4,2020/01/08 16:36:00+00,1636,300 Feet From,2.020001e+09,VIRGINIA BEACH,WED,NOT AT INTERSECTION,DUSK,ON ROADWAY,2400 BLK N. LANDING RD,...,REAR END,NO ADVERSE CONDITION (CLEAR/CLOUDY),NaN,NO,NaN,NaN,na,6,36.751716,-76.053755


In [3]:
data.dropna(subset=['latitude', 'longitude'], inplace=True)

In [4]:
import geopandas as gpd

In [5]:
gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.longitude, data.latitude), crs="EPSG:4326"
)

In [ ]:
gdf.explore()

In [ ]:
# Comparing haversine and geodesic distances
# Sources online claim that haversine is faster, but since haversine is designed for spherical surfaces, it is not as accurate.
# Geodesic is a more accurate measurement, but supposedly not as fast

In [9]:
from sklearn.metrics.pairwise import haversine_distances

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html
def haversine(point1, point2):
    result = haversine_distances([point1, point2])
    result *= 63780  # multiply by Earth's radius to get meters
    distance = (result[0, 1]**2 + result[1, 0]**2)**0.5
    return distance

from geopy.distance import distance as gpydistance

def geodesic(point1, point2):
  return gpydistance(point1, point2).meters

import time

point1 = gdf[['latitude', 'longitude']].iloc[0]
point1.latitude = 38.031387227679765
point1.longitude = -78.50257432749238
point2 = point1.copy()
point2.latitude = 38.03152372279175
point2.longitude = -78.50340559417177

print(f"Point 1: ({point1.latitude}, {point1.longitude})")
print(f"Point 2: ({point2.latitude}, {point2.longitude})")

start = time.time()
haversine_calculation = haversine(point1, point2)
elapsed = time.time() - start
print(f"Haversine Distance calculated: {haversine_calculation} in {elapsed}ms")

start = time.time()
geodesic_distance = geodesic(point1, point2)
elapsed = time.time() - start
print(f"GeoPy (geodesic) Distance calculated: {geodesic_distance} in {elapsed}ms")

# https://www.google.com/maps/place/38%C2%B001'53.5%22N+78%C2%B030'12.3%22W/@38.0315187,-78.5031495,20.5z/data=!4m4!3m3!8m2!3d38.0315237!4d-78.5034056?entry=ttu
print("Google Maps says 74.39 meters") 

Point 1: (38.031387227679765, -78.50257432749238)
Point 2: (38.03152372279175, -78.50340559417177)
Haversine Distance calculated: 71.9375722784385 in 0.0010006427764892578ms
GeoPy (geodesic) Distance calculated: 74.53701011818934 in 0.0ms
Google Maps says 74.39 meters


In [ ]:
# Geodesic seems to perform better, so we will go with that

In [10]:
from sklearn.cluster import DBSCAN
from geopy.distance import distance as gpydistance

epsilon = 25 # meters

def earth_distance(point1, point2):
  return gpydistance(point1, point2).m

# Takes a few minutes to run
dbscan = DBSCAN(eps=epsilon, min_samples=1, metric=earth_distance)
gdf['cluster'] = dbscan.fit_predict(gdf[['latitude', 'longitude']])

In [11]:
gdf["cluster"].value_counts()

cluster
58      90
27      76
14      73
72      71
89      71
        ..
1939     1
1940     1
1942     1
1944     1
3870     1
Name: count, Length: 3871, dtype: int64

In [24]:
index_of_interest = 27

In [25]:
gdf[["Main_Street", "Nearest_Street"]][gdf["cluster"] == index_of_interest]

,Main_Street,Nearest_Street
28,5300 INDIAN RIVER RD,5300 INDIAN RIVER RD
42,5300 INDIAN RIVER RD,5300 INDIAN RIVER RD
529,5300 INDIAN RIVER RD,5300 KEMPS RIVER RD
530,5300 INDIAN RIVER RD,5300 KEMPS RIVER RD
614,5300 INDIAN RIVER RD,KEMPSRIVER DR
...,...,...
8381,5300 INDIAN RIVER RD,KEMPSVILLE RD
8485,5300 INDIAN RIVER RD,KEMPSVILLE RD
8511,5300 INDIAN RIVER RD,1400 KEMPSVILLE RD
8661,5300 INDIAN RIVER RD,KEMPSVILLE RD


In [ ]:
gdf[gdf["cluster"] == index_of_interest].explore()

In [28]:
gdf.to_file('data/gdf_with_clusters.shp')

C:\Users\andre\AppData\Local\Temp\ipykernel_11332\2806781876.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file('data/gdf_with_clusters.shp')
